In [1]:
PROJECT = "RFCX"
EXP_NUM = "0"
EXP_TITLE = "Baseline"
EXP_NAME = "exp_" + EXP_NUM + "_" + EXP_TITLE
IS_WRITRE_LOG = True
print('expname:' + EXP_NAME)

In [2]:
# library import
import numpy as np
import pandas as pd
import os
import tqdm
import random
import time
import math
import csv
import torch
import torch.nn as nn
from time import sleep
from torch.nn import functional as F
from torch.optim import Adam, AdamW
from torchvision.models import resnet18, resnet34, resnet50
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
import torch.utils.data as torchdata
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from contextlib import contextmanager
from typing import Optional
import logging
from numpy.random import beta
from pathlib import Path

from fastprogress.fastprogress import master_bar, progress_bar

from torchviz import make_dot


from conformer import ConformerConvModule
from conformer import ConformerBlock

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

from torch.utils.tensorboard import SummaryWriter

import wandb


pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [3]:
# use GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [4]:
# sample data
sample = torch.from_numpy(np.load('melspec.npy'))
print(sample.shape)
print(sample[np.newaxis, :, :].shape)
# channel = sample.unsqueeze(0)
# batch = channel.unsqueeze(0)
# print(batch.shape)

In [5]:
params = {
    "SEED":               42,
    "INPUT":              Path("../input/rfcx-species-audio-detection/train"),
    "TRAIN_AUDIO_ROOT":   Path("../input/rfcx-species-audio-detection/train_mel"),
    "TEST_AUDIO_ROOT":    Path("../input/rfcx-species-audio-detection/test_mel"),
    "TRAIN_TP":           Path("../input/rfcx-species-audio-detection/train_tp.csv"),
    "SUB":                Path("../input/rfcx-species-audio-detection/sample_submission.csv"),
    "DIM":                sample.shape[1],
    "SEQ_LEN":            sample.shape[0],
    "KERNEL_SIZE":        3,
    "POOL_SIZE":          2,
    "POOL_STRIDE":        2,
    "NUM_BIRDS":          24,
    "N_FOLDS":            5,
    "BATCH_NUM":          50,
    "VALID_BATCH_NUM":    50,
    "EPOCH_NUM":          50,
    "DROPOUT":            0.35,
    "lr":                 0.0001,
    "eta_min":            1e-5,
    "t_max":              10,
}
wandb.init(config=params, project=PROJECT, group=EXP_NAME)
config = wandb.config

print(config.SEED)

In [6]:
params = {
    "SEED":               42,
    "INPUT":              Path("../input/rfcx-species-audio-detection/train"),
    "TRAIN_AUDIO_ROOT":   Path("../input/rfcx-species-audio-detection/train_mel"),
    "TEST_AUDIO_ROOT":    Path("../input/rfcx-species-audio-detection/test_mel"),
    "TRAIN_TP":           Path("../input/rfcx-species-audio-detection/train_tp.csv"),
    "SUB":                Path("../input/rfcx-species-audio-detection/sample_submission.csv"),
    "DIM":                sample.shape[1],
    "SEQ_LEN":            sample.shape[0],
    "KERNEL_SIZE":        3,
    "POOL_SIZE":          2,
    "POOL_STRIDE":        2,
    "NUM_BIRDS":          24,
    "N_FOLDS":            5,
    "BATCH_NUM":          50,
    "VALID_BATCH_NUM":    50,
    "EPOCH_NUM":          50,
    "DROPOUT":            0.35,
    "lr":                 0.0001,
    "eta_min":            1e-5,
    "t_max":              10,
}
wandb.init(config=params, project=PROJECT)
config = wandb.config
wandb.run.name = EXP_NAME
wandb.run.save()
print(config.SEED)